# Find top 10000 vocab + Logistic Rgression

End of week 10: tasks 0-2

In [ ]:
import re
import numpy as np
import pandas as pd
import nltk
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import requests
import ast

In [14]:
# Mapping af typer til "fake" eller "reliable"
type_mapping = {
    'unreliable': 'fake',
    'fake': 'fake',
    'conspiracy': 'fake',
    'bias': 'fake',
    'junksci': 'fake',
    'clickbait': 'reliable',
    'reliable' : 'reliable',
    'state': 'fake',
    'political': 'reliable',
    'satire': 'fake',
    'hate': 'fake',
    'rumor': 'fake',
}

In [21]:
train_data = pd.read_csv("15_000_preprocessed_train.csv")

# Content is loaded as string type. Set type to list of string again.
train_data['content'] = train_data['content'].apply(ast.literal_eval)

In [ ]:
# temporary delete 
train_data = train_data.drop(train_data[train_data['type'] == 'unknown'].index)

In [ ]:
# Mapper "type" til en ny "label"-kolonne
train_data["label"] = train_data["type"].map(type_mapping)

In [59]:
train_data = train_data.dropna(subset=["label"])
train_data['content'] = train_data['content'].fillna('').map

In [61]:
# Optælling af ord
combined_list = train_data['content'].explode().tolist()

In [62]:
import collections
word_freq = collections.Counter(combined_list)

top_10000 =word_freq.most_common(10000)

In [63]:
# ---- 1. Indlæs og forbered data ----

# Top 10.000 ord

top_10000_words = [word for word, _ in top_10000]

df_top_words = pd.DataFrame(top_10000_words, columns=['Word'])
df_top_words.to_csv("top_10000_vocabulary.csv", index=False)


# Logistic Regression

In [64]:
from sklearn.preprocessing import MaxAbsScaler
import re
import numpy as np
import pandas as pd
import nltk
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [81]:
# ---- 5. Konverter tekst til Bag-of-Words feature matrix ----
df_top_words = pd.read_csv("top_10000_vocabulary.csv")
top_10000_words = df_top_words['Word'].tolist()
vectorizer = CountVectorizer(vocabulary = top_10000_words)
vectorizer.fit(top_10000_words)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:1368: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


CountVectorizer(vocabulary=['<bound method Series.map of 0        [waldman, '
                            '--, carolyn, evelyn, (, greiner, ), ...\n'
                            '1        [portugues, boss, line, take, loui, van, '
                            'gaal,...\n'
                            '2        [chri, daugherti, ,, highway, worker, '
                            'ohio, ,,...\n'
                            '3        [make, music, wild, primit, forc, simpl, '
                            'matte...\n'
                            '4        [intern, subsidi, cut, iran, fuel, '
                            'price, soar...\n'
                            '                               '
                            '...                        \n'
                            '10879    [barenboim, la, scala, seal, deal, '
                            'teatro, all...\n'
                            "10880    [weekend, recap, (, 's, news, ), "
                            'saturday, mor...\n'
                            '10881    [jeffrey, archer, ,, millionair, '
                            'novelist, for...\n'
                            "10882    [page, categori, ``, ocean, '', follow, "
                            '<num>,...\n'
                            '10883    [first, ,, so-cal, ``, religi, liberti, '
                            "'', al...\n"
                            'Name: content, Length: 10374, dtype: object>'])

In [ ]:
# Transformer træning, valid og test data
# train_data['content'] = train_data['content'].fillna('')

# X_train = vectorizer.transform(train_data['content'])
X_train = vectorizer.transform(train_data['content'].fillna('').astype(str))


In [78]:


# Konverter til tæt matrix (da StandardScaler ikke understøtter sparse matrices direkte)
# X_train_dense = X_train.toarray()
# X_valid_dense = X_valid.toarray()
# X_test_dense = X_test.toarray()

scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Labels
y_train = train_data['label']

# ---- 6. Træn Logistic Regression model ----
clf = LogisticRegression(max_iter=100000, solver='saga', random_state=42)
clf.fit(X_train_scaled, y_train)

# ---- 7. Evaluer modellen ----
y_pred = clf.predict(X_train_scaled)

# Udskriv resultater
print(classification_report(y_train, y_pred))

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train, y_pred))


              precision    recall  f1-score   support

        fake       0.52      1.00      0.68      5391
    reliable       0.00      0.00      0.00      4983

    accuracy                           0.52     10374
   macro avg       0.26      0.50      0.34     10374
weighted avg       0.27      0.52      0.36     10374

[[5391    0]
 [4983    0]]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# # ---- 5. Indlæs top 10.000 ord fra CSV og konverter tekst til Bag-of-Words feature matrix ----

# # Indlæs ordforrådet fra CSV
# df_vocab = pd.read_csv("top_10000_vocabulary.csv")
# top_10000_words = df_vocab["Word"].tolist()  # Konverter til liste

# # Opret CountVectorizer med det indlæste ordforråd
# vectorizer = CountVectorizer(vocabulary=top_10000_words) # BoW bag of words

# # Transformer træning, valid og test data
# X_train = vectorizer.transform(train['content'])
# X_valid = vectorizer.transform(valid['content'])
# X_test = vectorizer.transform(test['content'])

# # Labels
# y_train = train['label']
# y_valid = valid['label']
# y_test = test['label']

# # ---- 6. Træn Logistic Regression model ----

# clf = LogisticRegression(max_iter=5000, solver='saga', random_state=42)
# clf.fit(X_train, y_train)

# # ---- 7. Evaluer modellen ----

# y_pred = clf.predict(X_test)

# # Udskriv resultater
# print(classification_report(y_test, y_pred))


# Kode kopieret fra YouTube: https://www.youtube.com/watch?v=aL21Y-u0SRs&t=212s

In [ ]:
# from matplotlib import pyplot as plt
# import numpy as np

# from sklearn.linear_model import LogisticRegression
# model = LogisticRegression()
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# print(model.score(X_test, y_test))

# from sklearn.metrics import confusion_matrix
# print(confusion_matrix(y_test, y_pred))
# from sklearn.metrics import classification_report
# print(classification_report(y_test, y_pred))

# Gammel version 1... 

In [ ]:
# import re
# import numpy as np
# import pandas as pd
# import nltk
# from collections import Counter
# from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize

# # Ensure NLTK resources are available
# # nltk.download('punkt')
# # nltk.download('stopwords')

# # Load data efficiently
# data = pd.read_csv("995,000_rows.csv", usecols=["content"], dtype=str)

# # Shuffle and split dataset (80% train, 10% validation, 10% test)
# train, valid, test = np.split(
#     data.sample(frac=1, random_state=42),  # Shuffle dataset
#     [int(0.8 * len(data)), int(0.9 * len(data))]  # Indices for 80% train, 10% valid, 10% test
# )

# # Precompile regex patterns
# date_regex = re.compile(r"""\b(
#     (?:\d{1,2}[./-]\d{1,2}[./-]\d{2,4}) |
#     (?:\d{4}[./-]\d{1,2}[./-]\d{1,2}) |
#     (?:\b\d{1,2}(st|nd|rd|th)?\s+(of\s+)?[A-Za-z]+\s+\d{4}\b) |
#     (?:\b[A-Za-z]+\s+\d{1,2}(st|nd|rd|th)?,?\s+\d{4}\b) |
#     (?:\b[A-Za-z]+\s+\d{4}\b)
# )\b""", re.VERBOSE | re.IGNORECASE)

# number_regex = re.compile(r'\d+')
# email_regex = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b')
# url_regex = re.compile(r'\b(?:http[s]?://|www\.)[^\s<>"]+\b')
# whitespace_regex = re.compile(r'\s+')

# # Function to clean text
# def clean_text(text):
#     text = text.lower()
#     text = whitespace_regex.sub(' ', text)
#     text = date_regex.sub("<DATE>", text)
#     text = url_regex.sub("<URL>", text)
#     text = email_regex.sub("<EMAIL>", text)
#     text = number_regex.sub("<NUM>", text)
#     return text

# # Apply text cleaning to each split
# train['content'] = train['content'].fillna('').map(clean_text)
# valid['content'] = valid['content'].fillna('').map(clean_text)
# test['content'] = test['content'].fillna('').map(clean_text)

# # Tokenization and stopword filtering (only on train set)
# stop_words = set(stopwords.words('english'))
# tokens = [
#     word for sentence in train['content'] 
#     for word in word_tokenize(sentence) 
#     if word.isalpha() and word not in stop_words
# ]

# # Stemming
# stemmer = PorterStemmer()
# stemmed_tokens = [stemmer.stem(word) for word in tokens]

# # Compute word frequencies
# word_counts = Counter(stemmed_tokens)

# # Extract top 10,000 words
# top_10000 = word_counts.most_common(10000)

# # Export to CSV
# df_top_words = pd.DataFrame(top_10000, columns=['Word', 'Frequency'])
# df_top_words.to_csv("top_10000_vocabulary.csv", index=False)

# print("Top 10,000 words saved to top_10000_vocabulary.csv")
